### This notebook is a quick exploration of the content and preprocessing of the raw s3p dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
df = pd.read_csv("../data/01_raw/s3p.csv", low_memory=False)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.columns.unique()

In [ ]:
df.describe()

In [ ]:
numerical = df.select_dtypes(include="number")
print("N° numerical cols:", len(numerical.columns))

In [ ]:
categorical = df.select_dtypes(exclude="number")
print("N° categorical cols:", len(categorical.columns))

In [ ]:
# explore cardinality in dataset
cardinal_cols = df.nunique().sort_values(ascending=False)
cardinal_cols

In [ ]:
#select cardinal columns to delete (columns > 80% uniques values)

card_to_del = cardinal_cols[cardinal_cols > 0.8 * len(df)].index
card_to_del

In [ ]:
#delete high cardinal columns. This columns doesn't good for create models

df.drop(columns=card_to_del, axis=1, inplace=True)
df.shape

In [ ]:
# explore unique values ​​of some interesting variables
df["tipoFalta"].unique()

In [ ]:
# show null values to take actions
nulls = df.isnull().sum()
nulls.sort_values(ascending=False)

In [ ]:
# delete columns with high null values (null > 70%)

cols_delete = nulls[nulls > 0.7 * len(df)].index
cols_delete

In [ ]:
# drop columns with many null values
df.drop(columns=cols_delete, axis=1, inplace=True)
df.shape

In [ ]:
df.sample(10)

In [ ]:
# explore large text columns 
df["objetoContrato"].unique()

In [ ]:
# explore not good parsed (json to csv) columns for take actions
df["tipoSancion"]

In the following, transformations and cleaning are carried out on the columns of interest

In [ ]:
df2 = df.copy()

In [ ]:
# rename columns, remove (".") dot structure created with json_normalize to avoid confusion
import re
columns = df2.columns

for column in columns:
    # replace . to _ using re library
    new_column = re.sub(r'\.', '_', column)
    df2 = df2.rename(columns={column: new_column})

df2.columns

In [ ]:
# drop not interesting columns. Names are not good for predictions,
# 'institucionDependencia_siglas' is high correlate to 'institucionDependencia_nombre'
# and 'resolucion_fechaNotificacion' is a field that does not provide significant 
# information in a low complexity analysis

not_interesting_cols = [
    "institucionDependencia_siglas",
    "responsableSancion_nombres",
    "responsableSancion_primerApellido",
    "responsableSancion_segundoApellido",
    "resolucion_fechaNotificacion"
]
df2.drop(not_interesting_cols, axis=1, inplace=True)
df2.shape


In [ ]:
df3 = df2.copy()

In [ ]:
df3.info()

In [ ]:
# formatting fechaCaptura column to exclude T HH:mm:ss

def removeTime(string:str):
    return string.split("T")[0]


df3["fechaCaptura"] = df3["fechaCaptura"].apply(removeTime)
df3["fechaCaptura"].unique()

In [ ]:
# fill null values in each column with itself mode value

for column in df3.columns:
    df3[column].fillna(df3[column].mode()[0], inplace=True)

df3.isnull().sum()

In [ ]:
#drop rows with '-' in inhabilitacion_fechaFinal and inhabilitacion_fechaInicial columns
df3.drop(df3[df3['inhabilitacion_fechaFinal'] == '-'].index, inplace=True)
df3.drop(df3[df3['inhabilitacion_fechaInicial'] == '-'].index, inplace=True)

In [ ]:
# function to standardize the datetime format across all columns with this date type
def formatDatetime(string:str):
    formats = ['%Y-%m-%d', '%d-%b-%Y', '%d-%m-%Y']
    for format in formats:
        try:
            return datetime.strptime(string, format).strftime('%Y/%m/%d')
        except Exception as e:
            return None

In [ ]:
# Format date type columns to the desired format
df3['inhabilitacion_fechaInicial'] = df3['inhabilitacion_fechaInicial'].apply(formatDatetime)
df3['inhabilitacion_fechaFinal'] = df3['inhabilitacion_fechaFinal'].apply(formatDatetime)

In [ ]:
# show a bug in conversion datetime format (anomaly in datetime to formatting '%d-%b-%Y', unresolved)
# rows with new null values in above re-formatted columns
#df3.info()

In [ ]:
# delete rows with new null values (anomaly in datetime to formatting '%d-%b-%Y')
df3.dropna(inplace=True)


In [ ]:
# reset index of dataframe
df3.reset_index(drop=True, inplace=True)

In [ ]:
# change datatype from date columns (object to datetype)
date_cols = [col for col in df3.columns if "fecha" in col]
df3[date_cols] = df3[date_cols].apply(pd.to_datetime)

In [ ]:
# Calculate the sanction duration in days ans assing it to new column 'tiempo_sancion'

df3["tiempo_sancion"] = (df3['inhabilitacion_fechaFinal'] - df3['inhabilitacion_fechaInicial']).dt.days
df3["tiempo_sancion"]

In [ ]:
df4 = df3.copy()

In [ ]:
# transform resolution_url column values to binary label (int type) according if It has url or no.

df4["resolucion_url"] = df4["resolucion_url"].apply(lambda x: 1 if "https:" in x else 0)
df4["resolucion_url"]

In [ ]:
df4.info()

In [ ]:
df4["tipoSancion"][1]

In [ ]:
# create mapper function from tipoSancion column

def mapperSanctions(string:str):
    if 'INHABILITA' in string and 'MULT' not in string:
        return 'inhabilitado'
    elif 'INHABILITA' not in string and 'MULT' in string:
        return 'multado'
    elif 'INHABILITA' in string and 'MULT' in string:
        return 'multado e inhabilitado'
    else:
        return string


df4["tipoSancion"] = df4["tipoSancion"].apply(mapperSanctions)
df4["tipoSancion"]

In [ ]:
df4.info()

In [ ]:
df4["tipoSancion"]

In [ ]:
# encoding categorical columns to compare and create analysis
df5 = pd.get_dummies(df4, sparse=True)

In [ ]:
df5.shape

In [ ]:
corr = df5.corr()
corr